Antes de pasar al código, es fundamental entender cómo realizamos las conexiones físicas. Para este proyecto utilicé dos sensores DS18B20 (el modelo que viene con recubrimiento metálico).

 El Cableado
Cada sensor cuenta con tres cables: Rojo (Positivo), Negro (Negativo) y Amarillo (Señal). Como utilicé dos sensores para medir diferentes puntos, realicé un empalme de cables en paralelo:

Uní los dos cables negros y los alargué con un cable DuPont hacia el pin GND.

Uní los dos cables rojos y los llevé al pin 3V3.

Uní los dos cables amarillos y los conecté al pin GPIO 34.

 La Resistencia de Pull-up
Entre el cable de Positivo (3V3) y el de Señal, instalé una resistencia de 4.7kΩ. La función de esta resistencia es evitar el "ruido" eléctrico y mantener firme la señal en el cable. Al darle energía constante a la línea de datos, aseguramos que el mensaje del sensor llegue limpio al ESP32 sin interferencias externas.

A continuación, se presentan dos versiones del código fuente. La lógica principal es la misma, pero cambia el destino de los datos:

Modo Test: Utiliza la Test URL de n8n. Es ideal para cuando estamos configurando el flujo y queremos ver los datos entrando en tiempo real en la pantalla de diseño de n8n.


In [ ]:
#include <WiFi.h>
#include <HTTPClient.h>

const char* ssid = "TU_WIFI";
const char* password = "TU_PASSWORD";
// URL con el prefijo -test
const char* serverName = "http://192.168.100.53:5678/webhook-test/lecturas-camara";

void setup() {
  Serial.begin(115200);
  WiFi.begin(ssid, password);
  while (WiFi.status() != WL_CONNECTED) { delay(500); Serial.print("."); }
  Serial.println("\nWiFi de prueba conectado");
}

void loop() {
  if (WiFi.status() == WL_CONNECTED) {
    HTTPClient http;
    http.begin(serverName);
    http.addHeader("Content-Type", "application/json");

    // Datos estáticos para verificar la ruta
    String json = "{\"sensor_id\": \"TEST_ESP32\", \"temp_interior\": 25.0, \"temp_exterior\": 25.0}";
    
    int httpResponseCode = http.POST(json);
    Serial.printf("Respuesta Test (debe ser 200): %d\n", httpResponseCode);
    http.end();
  }
  delay(10000); // 10 segundos entre pruebas
}

Modo Producción: Utiliza la Production URL y la IP fija de la Raspberry Pi. Este es el código definitivo que permite que el sistema funcione de forma autónoma las 24 horas sin necesidad de tener el navegador abierto.

In [ ]:
#include <WiFi.h>
#include <HTTPClient.h>
#include <OneWire.h>
#include <DallasTemperature.h>

// Configuración de Red
const char* ssid = "TU_WIFI_REAL";
const char* password = "TU_PASSWORD_REAL";
// NOTA: Se usa la IP 192.168.100.53 porque el ESP32 no resuelve .local
const char* serverName = "http://192.168.100.53:5678/webhook/lecturas-camara";

// Configuración de Sensores
const int oneWireBus = 4; // Pin GPIO4
OneWire oneWire(oneWireBus);
DallasTemperature sensors(&oneWire);

void setup() {
  Serial.begin(115200);
  sensors.begin();
  WiFi.begin(ssid, password);
  while (WiFi.status() != WL_CONNECTED) { delay(500); }
}

void loop() {
  if (WiFi.status() == WL_CONNECTED) {
    sensors.requestTemperatures(); 
    float tempInt = sensors.getTempCByIndex(0);
    float tempExt = sensors.getTempCByIndex(1);

    // Construcción del JSON dinámico
    String json = "{\"sensor_id\": \"ESP32_CAMARA_01\", \"temp_interior\": " 
                   + String(tempInt) + ", \"temp_exterior\": " + String(tempExt) + "}";

    HTTPClient http;
    http.begin(serverName);
    http.addHeader("Content-Type", "application/json");

    int httpResponseCode = http.POST(json);
    
    // Log de éxito o error
    if (httpResponseCode > 0) {
      Serial.printf("Dato enviado. Código HTTP: %d\n", httpResponseCode);
    } else {
      Serial.printf("Error en conexión: %s\n", http.errorToString(httpResponseCode).c_str());
    }
    http.end();
  }
  
  // Frecuencia de muestreo: 1 minuto (60000 ms)
  delay(60000); 
}

Para corroborar que todo el flujo de datos se ha completado correctamente y que la información ya reside en nuestro servidor, podemos reutilizar el Workflow 1 (el que usamos inicialmente para crear la tabla).

En lugar de ejecutar el comando de creación, cambiaremos la consulta en el nodo de PostgreSQL por un código que nos permita visualizar los valores almacenados. Simplemente borra el contenido anterior y pega lo siguiente:
(Nota: Asegúrate de usar el nombre exacto que le diste a tu tabla; en mi caso es lecturas_sensor).

In [ ]:
SELECT * FROM lecturas_sensor ORDER BY fecha_registro DESC LIMIT 20;